In [3]:
# coding: utf-8
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local[*]', 'pyspark')


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark, master=local[*]) created by __init__ at <ipython-input-1-80ed690d12eb>:6 

In [4]:
import numpy
import csv
from Similarities import cosine_similarity
top="33173 33475 1076 35300 15743"
train_rdd = sc.textFile("train_no_header.csv")
test_rdd = sc.textFile("test_no_header.csv")
def test_item_filter(x):
    for i in x[1]:
        if i[0] in test_users:
            return True
    return False

def map_string (x):
        y=x[1]
        s=""
        
        for i in y:
            s+=str(i[0])+" "
        z=len(y)
        top_rates=[33173, 33475, 1076, 35300, 15743]
        i=0
        while (z<5):
            s+=str(top_rates[i])
            i+=1
            z+=1
        if (y[0][1]<8):
            s=top
        return x[0],s


def compute_rec (x):
    num=sum(i[0]*i[1] for i in x[1])
    den=sum(i[1] for i in x[1])
    if den==0:
        return 0
    return x[0][0],(x[0][1],float(num)/den)

i=1800
k=200
while (i<4000):
    f1= open('submission_item_based'+str(i)+'.csv','w+')
    
    print "FILE: from "+str(i)+" to"+str(i+k)
    test_users=test_rdd.collect()[i:i+k]
    print test_users
    test_item_votes_rdd=(train_rdd
                     .map(lambda x: x.split(','))
                     .map(lambda x:(x[1],[x[0],int(x[2])]))
                     .groupByKey()
                     .filter(test_item_filter)
                     .map(lambda x: (x[0],list(x[1]))))
                          

    rdd = train_rdd
    rdd = rdd.map( lambda x: x.split(',') )
    rdd = rdd.map(lambda x:(x[1],[x[0],int(x[2])]))

    rdd = rdd.groupByKey()
    rdd = rdd.map(lambda x:(x[0],list(x[1])))



    #computing similarities
    

    sim_rdd=test_item_votes_rdd.cartesian(rdd)
    sim_rdd=sim_rdd.map(lambda x: (x[0][0],[x[1][0],cosine_similarity(x[1][1],x[0][1])])).filter(lambda x:x[1][1]>0 and x[1][0]!=x[0])

    #computing useful part of reverse urm 
    rev_urm_rdd=(train_rdd
         .map(lambda x: x.split(','))
         .filter(lambda x:x[0] in test_users)
         .map(lambda x:(x[1],[x[0],int(x[2])])))

    #grouping votes
    vote_rdd=rev_urm_rdd.join(sim_rdd)
    grouped_vote_rdd=(vote_rdd
                    .map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
                    .groupByKey())

    #computing recommendations
    rec_rdd=(grouped_vote_rdd
         .map(compute_rec)
         .groupByKey()
         .map(lambda x: (x[0],(sorted(list(y for y in x[1]),key=lambda x: -x[1]))[0:5])))

    
    #printing on file
    
    
    
    
    recommendations_rdd=(rec_rdd
                     .map(map_string)
                     .takeOrdered(k+1,key=lambda x: int(x[0])))
                    
    recommended_users=list(item[0] for item in recommendations_rdd)
    formatted_recommendations=recommendations_rdd;
    csv_f1=csv.writer(f1)
    m=0
    n=0
    while(m<k):
        if (test_users[m] in recommended_users):
            csv_f1.writerow(formatted_recommendations[n])
            n+=1
        else:
            csv_f1.writerow((test_users[m],top))
        m+=1
        
    i=i+k
    f1.close()
    



FILE: from 1800 to2000
[u'6642', u'6644', u'6645', u'6653', u'6654', u'6657', u'6666', u'6671', u'6674', u'6676', u'6678', u'6684', u'6687', u'6688', u'6698', u'6699', u'6700', u'6701', u'6703', u'6707', u'6708', u'6710', u'6716', u'6717', u'6728', u'6732', u'6735', u'6738', u'6739', u'6742', u'6751', u'6763', u'6777', u'6784', u'6795', u'6797', u'6798', u'6799', u'6802', u'6804', u'6807', u'6808', u'6817', u'6818', u'6819', u'6826', u'6827', u'6828', u'6832', u'6833', u'6834', u'6836', u'6844', u'6846', u'6851', u'6854', u'6858', u'6860', u'6866', u'6867', u'6869', u'6870', u'6874', u'6879', u'6882', u'6887', u'6889', u'6890', u'6893', u'6897', u'6907', u'6910', u'6914', u'6922', u'6932', u'6937', u'6939', u'6941', u'6950', u'6951', u'6952', u'6958', u'6961', u'6963', u'6966', u'6969', u'6972', u'6982', u'6985', u'6986', u'6987', u'6988', u'6992', u'6999', u'7003', u'7004', u'7005', u'7006', u'7008', u'7010', u'7012', u'7014', u'7015', u'7018', u'7022', u'7026', u'7030', u'7031', u'70

In [7]:
#writing last file chunk
import numpy
import csv
from Similarities import cosine_similarity
if True:
    f1= open('submission_item_based4000.csv','w+')
    test_users=test_rdd.collect()[4000:4196]
    print test_users
    test_item_votes_rdd=(train_rdd
                     .map(lambda x: x.split(','))
                     .map(lambda x:(x[1],[x[0],int(x[2])]))
                     .groupByKey()
                     .filter(test_item_filter)
                     .map(lambda x: (x[0],list(x[1]))))
                          

    rdd = train_rdd
    rdd = rdd.map( lambda x: x.split(',') )
    rdd = rdd.map(lambda x:(x[1],[x[0],int(x[2])]))

    rdd = rdd.groupByKey()
    rdd = rdd.map(lambda x:(x[0],list(x[1])))



    #computing similarities
    

    sim_rdd=test_item_votes_rdd.cartesian(rdd)
    sim_rdd=sim_rdd.map(lambda x: (x[0][0],[x[1][0],cosine_similarity(x[1][1],x[0][1])])).filter(lambda x:x[1][1]>0 and x[1][0]!=x[0])

    #computing useful part of reverse urm 
    rev_urm_rdd=(train_rdd
         .map(lambda x: x.split(','))
         .filter(lambda x:x[0] in test_users)
         .map(lambda x:(x[1],[x[0],int(x[2])])))

    #grouping votes
    vote_rdd=rev_urm_rdd.join(sim_rdd)
    grouped_vote_rdd=(vote_rdd
                    .map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
                    .groupByKey())

    #computing recommendations
    rec_rdd=(grouped_vote_rdd
         .map(compute_rec)
         .groupByKey()
         .map(lambda x: (x[0],(sorted(list(y for y in x[1]),key=lambda x: -x[1]))[0:5])))

    
    #printing on file
    
    
    top="33173 33475 35300 1076 31992"
    
    recommendations_rdd=(rec_rdd
                     .map(map_string)
                     .takeOrdered(196,key=lambda x: int(x[0])))
                    
    recommended_users=list(item[0] for item in recommendations_rdd)
    formatted_recommendations=recommendations_rdd;
    csv_f1=csv.writer(f1)
    m=0
    n=0
    while(m<196):
        if (test_users[m] in recommended_users):
            csv_f1.writerow(formatted_recommendations[n])
            n+=1
        else:
            csv_f1.writerow((test_users[m],top))
        m+=1
    f1.close()

[u'14703', u'14708', u'14709', u'14715', u'14716', u'14718', u'14720', u'14723', u'14726', u'14727', u'14728', u'14730', u'14733', u'14737', u'14738', u'14740', u'14749', u'14750', u'14751', u'14752', u'14756', u'14757', u'14760', u'14766', u'14769', u'14773', u'14775', u'14780', u'14781', u'14785', u'14787', u'14788', u'14789', u'14792', u'14794', u'14798', u'14801', u'14803', u'14809', u'14813', u'14815', u'14821', u'14833', u'14835', u'14843', u'14846', u'14851', u'14852', u'14854', u'14856', u'14859', u'14863', u'14864', u'14866', u'14869', u'14870', u'14873', u'14874', u'14876', u'14879', u'14883', u'14884', u'14890', u'14891', u'14893', u'14899', u'14908', u'14910', u'14911', u'14915', u'14917', u'14923', u'14926', u'14928', u'14930', u'14937', u'14939', u'14943', u'14947', u'14949', u'14950', u'14951', u'14952', u'14953', u'14955', u'14960', u'14970', u'14973', u'14978', u'14980', u'14985', u'14987', u'14988', u'14991', u'14993', u'14998', u'14999', u'15003', u'15008', u'15011',

In [9]:
#collecting results
i=0
rows=[["userId","testItems"]]
while(i<4001):
    f1= open('submission_item_based'+str(i)+'.csv','rb')
    f1_csv=csv.reader(f1)
    rows=rows+list(f1_csv)
    i=i+200
    f1.close()
f1=open('submission_item_based_cosine.csv','w+')
f1_csv=csv.writer(f1)
f1_csv.writerows(rows)
f1.close()